In [9]:
import numpy as np
import time
import sys
from collections import deque
import random

class Node:
    def __init__(self, state, parent=None):
        self.state = np.array(state, dtype=np.uint8)
        self.parent = parent
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1
    
    def __eq__(self, other):
        return np.array_equal(self.state, other.state)
    
    def __hash__(self):
        return hash(self.state.tobytes())
    
    def __str__(self):
        return str(self.state)
    
    def get_path(self):
        path = []
        current = self
        while current:
            path.append(current)
            current = current.parent
        return path[::-1]

In [10]:
def generate_random_puzzle():
    numbers = list(range(9))
    random.shuffle(numbers)
    return np.array(numbers, dtype=np.uint8).reshape(3, 3)

def is_solvable(puzzle):
    # Flatten the puzzle and remove the 0 (blank)
    flat = puzzle.flatten()
    flat = flat[flat != 0]
    
    # Count inversions
    inversions = 0
    for i in range(len(flat)):
        for j in range(i + 1, len(flat)):
            if flat[i] > flat[j]:
                inversions += 1
                
    # For a 3x3 puzzle, it's solvable if inversions are even
    return inversions % 2 == 0

In [11]:
def get_valid_moves(puzzle):
    moves = []
    blank_pos = np.where(puzzle == 0)
    row, col = blank_pos[0][0], blank_pos[1][0]
    
    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            moves.append((new_row, new_col))
    
    return moves, (row, col)

def apply_move(puzzle, blank_pos, move_pos):
    new_puzzle = puzzle.copy()
    new_puzzle[blank_pos], new_puzzle[move_pos] = new_puzzle[move_pos], new_puzzle[blank_pos]
    return new_puzzle

In [12]:
def solve_puzzle_dfs(initial_state, goal_state, max_depth=50):
    if not is_solvable(initial_state):
        return None
    
    goal_node = Node(goal_state)
    visited = set()
    stack = []
    
    initial_node = Node(initial_state)
    stack.append(initial_node)
    
    while stack:
        current_node = stack.pop()
        
        if current_node == goal_node:
            return current_node.get_path()
        
        if current_node.depth >= max_depth:
            continue
        
        if current_node not in visited:
            visited.add(current_node)
            
            moves, blank_pos = get_valid_moves(current_node.state)
            for move in moves:
                new_state = apply_move(current_node.state, blank_pos, move)
                new_node = Node(new_state, current_node)
                if new_node not in visited:
                    stack.append(new_node)
    
    return None

In [13]:
def print_puzzle(puzzle):
    for row in puzzle:
        print(" ".join(f"{num:2}" if num != 0 else "  " for num in row))
    print()

In [14]:
def main():
    # Generate solvable puzzle
    while True:
        initial_puzzle = generate_random_puzzle()
        if is_solvable(initial_puzzle):
            break
    
    goal_puzzle = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 0]], dtype=np.uint8)
    
    print("Initial puzzle:")
    print_puzzle(initial_puzzle)
    
    start_time = time.time()
    
    solution_path = solve_puzzle_dfs(initial_puzzle, goal_puzzle)
    
    end_time = time.time()
    
    if solution_path:
        print(f"Solution found in {len(solution_path)-1} moves!")
        print("\nSolution path:")
        for i, node in enumerate(solution_path):
            print(f"Step {i}:")
            print_puzzle(node.state)
    else:
        print("No solution found (might have exceeded max depth)")
    
    print(f"Time taken: {end_time - start_time:.4f} seconds")

if __name__ == "__main__":
    main()

Initial puzzle:
 8  3  5
 7     6
 2  1  4

Solution found in 48 moves!

Solution path:
Step 0:
 8  3  5
 7     6
 2  1  4

Step 1:
 8  3  5
 7  6   
 2  1  4

Step 2:
 8  3  5
 7  6  4
 2  1   

Step 3:
 8  3  5
 7  6  4
 2     1

Step 4:
 8  3  5
 7  6  4
    2  1

Step 5:
 8  3  5
    6  4
 7  2  1

Step 6:
 8  3  5
 6     4
 7  2  1

Step 7:
 8  3  5
 6  4   
 7  2  1

Step 8:
 8  3  5
 6  4  1
 7  2   

Step 9:
 8  3  5
 6  4  1
 7     2

Step 10:
 8  3  5
 6  4  1
    7  2

Step 11:
 8  3  5
    4  1
 6  7  2

Step 12:
 8  3  5
 4     1
 6  7  2

Step 13:
 8  3  5
 4  1   
 6  7  2

Step 14:
 8  3   
 4  1  5
 6  7  2

Step 15:
 8     3
 4  1  5
 6  7  2

Step 16:
    8  3
 4  1  5
 6  7  2

Step 17:
 4  8  3
    1  5
 6  7  2

Step 18:
 4  8  3
 1     5
 6  7  2

Step 19:
 4  8  3
 1  5   
 6  7  2

Step 20:
 4  8   
 1  5  3
 6  7  2

Step 21:
 4     8
 1  5  3
 6  7  2

Step 22:
    4  8
 1  5  3
 6  7  2

Step 23:
 1  4  8
    5  3
 6  7  2

Step 24:
 1  4  8
 5     3
 6  7  

In [15]:
# Just run this cell to execute the puzzle solver
main()

Initial puzzle:
 3  4  1
 7  8  5
 6     2

Solution found in 49 moves!

Solution path:
Step 0:
 3  4  1
 7  8  5
 6     2

Step 1:
 3  4  1
 7  8  5
 6  2   

Step 2:
 3  4  1
 7  8   
 6  2  5

Step 3:
 3  4  1
 7     8
 6  2  5

Step 4:
 3  4  1
    7  8
 6  2  5

Step 5:
 3  4  1
 6  7  8
    2  5

Step 6:
 3  4  1
 6  7  8
 2     5

Step 7:
 3  4  1
 6  7  8
 2  5   

Step 8:
 3  4  1
 6  7   
 2  5  8

Step 9:
 3  4  1
 6     7
 2  5  8

Step 10:
 3  4  1
    6  7
 2  5  8

Step 11:
 3  4  1
 2  6  7
    5  8

Step 12:
 3  4  1
 2  6  7
 5     8

Step 13:
 3  4  1
 2  6  7
 5  8   

Step 14:
 3  4  1
 2  6   
 5  8  7

Step 15:
 3  4  1
 2     6
 5  8  7

Step 16:
 3  4  1
    2  6
 5  8  7

Step 17:
 3  4  1
 5  2  6
    8  7

Step 18:
 3  4  1
 5  2  6
 8     7

Step 19:
 3  4  1
 5  2  6
 8  7   

Step 20:
 3  4  1
 5  2   
 8  7  6

Step 21:
 3  4  1
 5     2
 8  7  6

Step 22:
 3  4  1
    5  2
 8  7  6

Step 23:
 3  4  1
 8  5  2
    7  6

Step 24:
 3  4  1
 8  5  2
 7     